# Document Summerization

Process:

1. Generate summary for first page
2. Refine summary with second page
3. Repeate #2 for all pages




### Prepare Enviroments for following

- Azure Form Recognizer Endpoint & Key
- Azure OpenAI endpoint & key

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

# Setup environment

In [2]:
from AzureOpenAIUtil.AzureFormRecognizer import AzureFormRecognizerRead
from AzureOpenAIUtil.Summerization import summarization

# File Directories 
# -- raw data
RAW_DATA_FOLDER= 'data/raw'
# -- extracted json file 
EXTRACTED_DATA_FOLDER = 'data/extracted'
# -- summary files
SUMMARY_FOLDER = 'data/summary'
# Azure Form Recognizer Read
azure_fr = AzureFormRecognizerRead()
# Azure OpenAI util for summerization
summary_eng = summarization(deployment_name=os.getenv('DEPLOYMENT_NAME'))


c:\Users\louisli\AppData\Local\miniconda3\envs\openai_0\Lib\site-packages\langchain\llms\openai.py:1038: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


# Run From Recognizer to OCR documents

In [3]:
azure_fr.extract_files(RAW_DATA_FOLDER, EXTRACTED_DATA_FOLDER)

Processing file: 2010.15980.pdf    write output to data/extracted\2010.15980.json
Processing file: 2101.00190.pdf    write output to data/extracted\2101.00190.json
Processing file: 2104.08691.pdf    write output to data/extracted\2104.08691.json
Processing file: 2110.08387.pdf    write output to data/extracted\2110.08387.json
Processing file: 2201.11903.pdf    write output to data/extracted\2201.11903.json
Processing file: 2203.11171.pdf    write output to data/extracted\2203.11171.json
Processing file: 2211.01910.pdf    write output to data/extracted\2211.01910.json
Processing file: 2212.10496.pdf    write output to data/extracted\2212.10496.json


# Generate Summaries for extracted documents

In this demo, we show how to generate one short summary for each document.

*ignore the warning of input being truncated may show in the output - it's caused by GPT2 tokenizer for sequence longer than 1024*

In [3]:
doc_sum = summary_eng.generate_summary_files(source_folder=EXTRACTED_DATA_FOLDER, destination_folder=SUMMARY_FOLDER)



Generate summary for file: 2010.15980.json


ValueError: OpenAIChat currently only supports single prompt, got ['Write a concise summary of the following:\n\n\n"AUTOPROMPT: Eliciting Knowledge from Language Models with Automatically Generated Prompts Yasaman Razeghi *< Taylor Shin* Robert L. Logan IV* Eric Wallace Sameer Singh University of California, Irvine *University of California, Berkeley {tshin1, yrazeghi, rlogan, sameer } @uci.edu ericwallace@berkeley.edu Abstract The remarkable success of pretrained lan- guage models has motivated the study of what kinds of knowledge these models learn dur- ing pretraining. Reformulating tasks as fill- in-the-blanks problems (e.g., cloze tests) is a natural approach for gauging such knowledge, however, its usage is limited by the manual effort and guesswork required to write suit- able prompts. To address this, we develop AUTOPROMPT, an automated method to cre- ate prompts for a diverse set of tasks, based on a gradient-guided search. Using AUTO- PROMPT, we show that masked language mod-